# Teste de Modelos
Nesse notebook nós testamos diferentes modelos de classificação do skit-learn para descobrir qual a melhor técnica para predição dos sinais em libras feitos pelo usuário.

## Funções Auxiliares
Import de bibliotecas e definição de funções para a importação de dados e testes de modelos.

In [ ]:
import pickle
import numpy as np

In [4]:
# importa os dados do pickle
X = pickle.load(open("X_selected_alphabet.data", "rb"))
y = pickle.load(open("y_selected_alphabet.data", "rb"))
alphabet = pickle.load(open("alphabet-laguardia.data", "rb"))

In [5]:
def translate_y(y, inv=False):
  result = []
  translate = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7,
    'i': 8,
    'j': 9,
    'k': 10,
    'l': 11,
    'm': 12,
    'n': 13,
    'o': 14,
    'p': 15,
    'q': 16,
    'r': 17,
    's': 18,
    't': 19,
    'u': 20,
    'v': 21,
    'w': 22,
    'x': 23,
    'y': 24,
    'z': 25,
  }
  translate_inv = {}
  for value in translate:
    translate_inv[str(translate[value])] = value
  
  if inv:
    return translate_inv[str(y)]

  for elem in y:
    if elem in translate:
      result.append(translate[elem])
    else:
      result.append(elem)
  return result

In [6]:
def matches_alphabet(input):
    matches = 0
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'i', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v']
    alphabet = ['a', 'o', 'e', 'r', 'n', 'd', 't', 'l', 'p', 'v', 'g', 'b', 'q']
    alphabet = ['a', 'b', 'd', 'e', 'g', 'l', 'n', 'o', 'p', 'q', 'r', 't', 'v']
    
    for i, letter in enumerate(alphabet):
        if letter == input[i]:
            matches += 1
    
    return matches

In [9]:
# filtra o vetor de testes `alhpabet` com apenas as letras inclusas no alfabeto selecionado
selected_alphabet = ['a', 'o', 'e', 'r', 'n', 'd', 't', 'l', 'p', 'v', 'g', 'b', 'q']
old_alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'i', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v']

new_alphabet = []
for i, letter in enumerate(old_alphabet):
    if letter in selected_alphabet:
        new_alphabet.append(alphabet[i])
new_alphabet = np.array(new_alphabet)

## Teste de Modelos
Testa uma gama de modelos de classificação do Skit-learn para descobrirmos quais trazem os melhores resultados de predição.

In [ ]:
# imports a range of classifier models from sktlearn
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

# creates a list of tuples containing the classifier models and their names
classifiers = [
    ("Decision Tree", tree.DecisionTreeClassifier()),
    ("Random Forest", RandomForestClassifier()),
    ("K Nearest Neighbors", KNeighborsClassifier(1)),
    ("Gaussian Naive Bayes", GaussianNB()),
    ("Support Vector Machine", SVC()),
    ("Logistic Regression", LogisticRegression()),
    ("Multi-Layer Perceptron", MLPClassifier()),
]

# for each classifier in the list of tuples, fit the model to the training data
for name, clf in classifiers:
    clf.fit(X, y)

In [10]:
for name, clf in classifiers:
    answers = []
    for letter in new_alphabet:
        answers.append(translate_y(
            clf.predict(letter.reshape(1, -1))[0], inv=True))
    
    print(f"{name} \t\t answers: {answers} \t\t matches: {matches_alphabet(answers)}")

Decision Tree 		 answers: ['a', 't', 'd', 'l', 'g', 'l', 'd', 'e', 'q', 'q', 'r', 'b', 'r'] 		 matches: 6
Random Forest 		 answers: ['a', 't', 'd', 'e', 'g', 'l', 'q', 'e', 'a', 'q', 'r', 't', 'r'] 		 matches: 8
K Nearest Neighbors 		 answers: ['a', 't', 'd', 'b', 'g', 'l', 'q', 'o', 'd', 'q', 'r', 'd', 'r'] 		 matches: 7
Gaussian Naive Bayes 		 answers: ['a', 't', 'd', 'b', 'l', 'l', 'q', 't', 'b', 'q', 'r', 't', 'r'] 		 matches: 6
Support Vector Machine 		 answers: ['a', 't', 'd', 'e', 'l', 'l', 'q', 'o', 'd', 'q', 'r', 'b', 'r'] 		 matches: 7
Logistic Regression 		 answers: ['a', 'b', 'd', 'e', 'l', 'l', 'q', 'o', 'l', 'q', 'r', 't', 'v'] 		 matches: 10
Multi-Layer Perceptron 		 answers: ['a', 'b', 'd', 'e', 'g', 'l', 'q', 'o', 'p', 'q', 'r', 't', 'v'] 		 matches: 12


## Cria e exporta o melhor modelo
Baseado nos testes anteriores, descobrimos quais seriam os modelos com melhores resultados: KNN, Perceptron e Linear Regression. Baseado nos testes em outro notebook, descobrimos qual seria a melhor combinação modelo - hiper-parâmetro para a predição de resultados. A partir disso, concluímos que o Logistic Regression com os hiper-parâmetros abaixo faz as melhores predições.
Nessa célula, criamos e treinamos o modelo e o exportamos através do Pickle, para utilizar no produto final.

In [ ]:
model = LogisticRegression(
    penalty='none',
    fit_intercept=False,
    class_weight='balanced',
    max_iter=100,
    solver='newton-cg',
    multi_class='ovr',
)
model.fit(X, y)

# testa o resultado para a letra a
model.predict(new_alphabet[0].reshape(1, -1))

In [103]:
# exporta o modelo para o arquivo
pickle.dump(model, open("model.data", "wb"))